In [1]:

import torch 
from torch import nn 
import yfinance as yf
import talib as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly
import pandas as pd 
from torch.utils.data import DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score



In [2]:
#Finance Indicators 
f_indicators = ["MACD", "RSI", "CMO"," MOM", "Bollinger Bands", "SMA"]
ticker= "BTC-USD"
start_date = "2023-01-01"
end_date = "2025-12-31"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
close_price = data["Close"].values.flatten()
data.head(10)
close_price


[*********************100%***********************]  1 of 1 completed


array([16625.08007812, 16688.47070312, 16679.85742188, ...,
       87843.984375  , 86143.7578125 , 84980.875     ], shape=(1083,))

In [3]:
data.isna().sum()

Price   Ticker 
Close   BTC-USD    0
High    BTC-USD    0
Low     BTC-USD    0
Open    BTC-USD    0
Volume  BTC-USD    0
dtype: int64

In [4]:
#Finance Indicators 
f_indicators = ["MACD", "RSI", "CMO"," MOM", "Bollinger Bands", "SMA"]
ticker= "BTC-USD"
start_date = "2023-01-01"
end_date = "2025-12-31"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)
close_price = data["Close"].values.flatten()
data.head(10)
close_price

# MACD (TA-Lib returns 3 arrays)
macd, signal, hist = ta.MACD(close_price, fastperiod=12, slowperiod=26, signalperiod=9)

# Add to DataFrame
data["MACD"] = macd
data["Signal"] = signal
data["Hist"] = hist

# RSI
data["RSI"] = ta.RSI(close_price, timeperiod=14)

# CMO
data["CMO"] = ta.CMO(close_price, timeperiod=14)

# MOM
data["MOM"] = ta.MOM(close_price, timeperiod=14)

# Bollinger Bands
upper, middle, lower = ta.BBANDS(close_price, timeperiod=14, nbdevup=2, nbdevdn=2, matype=0)

# Add to DataFrame
data["BBU"] = upper    # Upper band
data["BBM"] = middle   # Middle band (SMA)
data["BBL"] = lower    # Lower band

# SMA
data["SMA"] = ta.SMA(close_price, timeperiod=14)
data.head(30)
data.isna().sum()

[*********************100%***********************]  1 of 1 completed


Price   Ticker 
Close   BTC-USD     0
High    BTC-USD     0
Low     BTC-USD     0
Open    BTC-USD     0
Volume  BTC-USD     0
MACD               33
Signal             33
Hist               33
RSI                14
CMO                14
MOM                14
BBU                13
BBM                13
BBL                13
SMA                13
dtype: int64

In [5]:
# Remove all rows with NaNs caused by indicator calculations
plot_data = data.dropna().copy()
plot_data = plot_data.copy()
plot_data.columns = [
    f"{lvl0}" if lvl1 else lvl0
    for lvl0, lvl1 in plot_data.columns
]

plot_data.index = pd.to_datetime(plot_data.index)  # ensure datetime index



In [6]:
plot_data

,Close,High,Low,Open,Volume,MACD,Signal,Hist,RSI,CMO,MOM,BBU,BBM,BBL,SMA
Date,,,,,,,,,,,,,,,
2023-02-03,23449.322266,23678.103516,23279.955078,23469.412109,27083066007,1658.371173,2013.486323,-355.115150,71.870229,43.740458,772.769531,23813.408952,23123.453404,22433.497856,23123.453404
2023-02-04,23331.847656,23556.949219,23291.794922,23446.320312,15639298538,1560.052550,1922.799569,-362.747019,70.054386,40.108773,554.222656,23832.374304,23163.040737,22493.707169,23163.040737
2023-02-05,22955.666016,23423.435547,22841.759766,23332.248047,19564262605,1435.235120,1825.286679,-390.051559,64.439754,28.879508,235.250000,23814.815753,23179.844308,22544.872863,23179.844308
2023-02-06,22760.109375,23119.279297,22692.025391,22954.021484,23825006542,1305.487771,1721.326897,-415.839126,61.672572,23.345143,-174.322266,23827.466311,23167.392718,22507.319124,23167.392718
2023-02-07,23264.291016,23310.974609,22756.257812,22757.267578,27187964471,1229.176130,1622.896744,-393.720614,65.755517,31.511035,627.822266,23803.674808,23212.237165,22620.799522,23212.237165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-12-14,88175.179688,90498.109375,87634.937500,90296.437500,50465972205,-1380.345062,-1765.137777,384.792715,40.374697,-19.250606,-2219.132812,94392.774509,90643.962612,86895.150714,90643.962612
2025-12-15,86419.781250,89983.921875,85304.078125,88171.078125,45559514323,-1606.842662,-1733.478754,126.636092,37.152956,-25.694087,98.210938,94367.639440,90650.977679,86934.315917,90650.977679
2025-12-16,87843.984375,88170.093750,85381.687500,86424.406250,41262178223,-1652.374805,-1717.257964,64.883160,41.249114,-17.501772,-3506.218750,94359.598823,90400.533482,86441.468141,90400.533482


In [7]:
# --- Create subplot layout ---
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_heights=[0.5, 0.25, 0.25],
    subplot_titles=[f"{ticker} Price & BB/SMA", "MACD", "RSI"]
)

# --- Row 1: Candlestick ---
fig.add_trace(
    go.Candlestick(
        x=plot_data.index,
        open=plot_data["Open"],
        high=plot_data["High"],
        low=plot_data["Low"],
        close=plot_data["Close"],
        name="Price"
    ),
    row=1, col=1
)

# Bollinger Bands
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBU"], name="BB Upper"), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBM"], name="BB Middle"), row=1, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["BBL"], name="BB Lower"), row=1, col=1)

# SMA
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["SMA"], name="SMA"), row=1, col=1)

# --- Row 2: MACD ---
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["MACD"], name="MACD"), row=2, col=1)
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["Signal"], name="Signal"), row=2, col=1)

# --- Row 3: RSI ---
fig.add_trace(go.Scatter(x=plot_data.index, y=plot_data["RSI"], name="RSI"), row=3, col=1)

# --- Layout ---
fig.update_layout(
    title=f"{ticker} Technical Analysis",
    height=900,
    template="plotly_dark",
    xaxis=dict(rangeslider=dict(visible=False))
)


# RSI reference lines
fig.update_yaxes(range=[0, 100], row=3, col=1)
fig.add_hline(y=70, line_dash="dash", line_color="red", row=3, col=1)
fig.add_hline(y=30, line_dash="dash", line_color="green", row=3, col=1)
fig.show()



In [8]:
plot_data.Close

Date
2023-02-03    23449.322266
2023-02-04    23331.847656
2023-02-05    22955.666016
2023-02-06    22760.109375
2023-02-07    23264.291016
                  ...     
2025-12-14    88175.179688
2025-12-15    86419.781250
2025-12-16    87843.984375
2025-12-17    86143.757812
2025-12-18    84980.875000
Name: Close, Length: 1050, dtype: float64

In [9]:
df = plot_data.copy()

df["label"] = np.where(df["Close"] < df["Close"].shift(1), "down","up")
df.loc[df.index[0],"Close"] = np.nan
df.label

Date
2023-02-03      up
2023-02-04    down
2023-02-05    down
2023-02-06    down
2023-02-07      up
              ... 
2025-12-14    down
2025-12-15    down
2025-12-16      up
2025-12-17    down
2025-12-18    down
Name: label, Length: 1050, dtype: object

In [10]:
features = ["MACD", "RSI", "CMO","MOM", "BBM", "SMA"]

label = "label"
df_input = df[features].copy()
df_output = df[label].copy()

#Transform dataset 
scaler = MinMaxScaler()
df_input[features] = scaler.fit_transform(df_input)
df_output = pd.DataFrame(np.where(df_output== "up", 1, 0), columns=["label"], index=df_output.index)

#Training data 
data_train_x, data_train_y = df_input.loc[:"2024-12-31", :], df_output.loc[:"2024-12-31", :]    
data_test_x, data_test_y = df_input.loc["2024-12-25":, :], df_output.loc["2024-12-25":, :]





In [11]:
data_test_y

,label
Date,
2024-12-25,1
2024-12-26,0
2024-12-27,0
2024-12-28,1
2024-12-29,0
...,...
2025-12-14,0
2025-12-15,0
2025-12-16,1


In [12]:
df_transformed = pd.concat([df_input, df_output], axis=1)
df_transformed.tail(360)

,MACD,RSI,CMO,MOM,BBM,SMA,label
Date,,,,,,,
2024-12-24,0.527516,0.486694,0.486694,0.491504,0.804062,0.804062,1
2024-12-25,0.527155,0.503631,0.503631,0.397613,0.802681,0.802681,1
2024-12-26,0.504532,0.402666,0.402666,0.340092,0.799550,0.799550,0
2024-12-27,0.476239,0.361299,0.361299,0.266256,0.794173,0.794173,0
2024-12-28,0.459992,0.392800,0.392800,0.292558,0.789597,0.789597,1
...,...,...,...,...,...,...,...
2025-12-14,0.350406,0.310256,0.310256,0.389246,0.706409,0.706409,0
2025-12-15,0.332952,0.263470,0.263470,0.445402,0.706481,0.706481,0
2025-12-16,0.329443,0.322955,0.322955,0.358056,0.703897,0.703897,1


In [13]:
df_output.values

array([[1],
       [0],
       [0],
       ...,
       [1],
       [0],
       [0]], shape=(1050, 1))

In [14]:
def create_dataset(data_x, data_y, window):
    data_x_u = []
    data_y_u = []
    data_y = data_y.reshape(-1)
    
    for i in range(window, len(data_x)):
        # Sequence input
        x_seq = torch.from_numpy(data_x[i - window: i, :]).unsqueeze(0).unsqueeze(0).float()
        data_x_u.append(x_seq)
        
        # Corresponding label as 1D tensor
        y_label = torch.tensor([data_y[i]]).long()
        data_y_u.append(y_label)
    
  
    X_tensor = torch.cat(data_x_u, dim=0)   
    y_tensor = torch.cat(data_y_u, dim=0)   
    
    return X_tensor, y_tensor
X_train, y_train = create_dataset(data_train_x.values, data_train_y.values, window= 6)
X_test, y_test = create_dataset(data_test_x.values, data_test_y.values, window= 6)



In [15]:
y_train.size()

torch.Size([692])

In [16]:
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)

dataset_loader_train = DataLoader(dataset_train, batch_size=24, shuffle=False)
dataset_loader_test = DataLoader(dataset_test, batch_size=24, shuffle=False)

In [17]:
#Modeling 
class CNNTA(nn.Module):
    def __init__(self, in_channels,  out_channels, input_size, kernel_size, out_features, dropout):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels= in_channels,  out_channels=out_channels, kernel_size=kernel_size),
            nn.BatchNorm2d(out_channels),
            nn.MaxPool2d(kernel_size=2),
            nn.ReLU()
        )
        with torch.no_grad():
            x = torch.zeros(1, in_channels, *input_size)
            x = self.layer1(x)
            flattened_size = int(np.prod(x.size()[1:]))
        
        #Fully connected Layer
        
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(in_features=flattened_size, out_features=out_features)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.layer1(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.classifier(x)
        return x 
        

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [19]:
torch.manual_seed(42)
model = CNNTA(1, 80, (6, 6), 2, 2, 0.3 )
model.to(device)



CNNTA(
  (layer1): Sequential(
    (0): Conv2d(1, 80, kernel_size=(2, 2), stride=(1, 1))
    (1): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): ReLU()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (dropout): Dropout(p=0.3, inplace=False)
  (classifier): Linear(in_features=320, out_features=2, bias=True)
)

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


In [21]:
torch.manual_seed(42)
epochs = 600

for epoch in range(epochs):
    # --- Training ---
    model.train()  # Only once per epoch
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch_x, batch_y in tqdm(dataset_loader_train, desc=f"Epoch {epoch+1}/{epochs} - Training", leave=False):
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        print(batch_y[:10])
        print(batch_y.shape)


        optimizer.zero_grad()
        predicted = model(batch_x)
        loss = criterion(predicted, batch_y)
        loss.backward()
        total_norm = 0
        for p in model.parameters():
            if p.grad is not None:
                total_norm += p.grad.data.norm(2).item() ** 2
        total_norm = total_norm ** 0.5
        print(f"Gradient norm: {total_norm}")
        optimizer.step()

        train_loss += loss.item() * batch_x.size(0)
        _, pred = torch.max(predicted, dim=1)
        correct_train += (pred == batch_y).sum().item()
        total_train += batch_x.size(0)

    avg_train_loss = train_loss / total_train
    train_accuracy = correct_train / total_train
    print(f"Epoch: {epoch+1}| Train Loss: {avg_train_loss: .3f}| Train Accuracy: {train_accuracy: .3f}")


Epoch 1/600 - Training:   0%|          | 0/29 [00:00<?, ?it/s]

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 4.710398310235161
tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1])
torch.Size([24])
Gradient norm: 1.6331106948780867
tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
torch.Size([24])
Gradient norm: 3.144309633771739
tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
torch.Size([24])
Gradient norm: 3.14106719628486
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])
torch.Size([24])
Gradient norm: 1.7548971144731464
tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
torch.Size([24])
Gradient norm: 1.6633271468077042
tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0])
torch.Size([24])
Gradient norm: 3.9731600039012522
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([24])
Gradient norm: 3.4174187274539825
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1])
torch.Size([24])
Gradient norm: 3.81456017411466
tensor([1, 1, 1, 0, 0, 1, 1, 0, 0, 1])
torch.Size([24])
Gradient norm: 1.6222535428738796
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 1.6332157580953213
tensor([1, 1, 0,

Epoch 2/600 - Training:   0%|          | 0/29 [00:00<?, ?it/s]

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 4.493967710242593
tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1])
torch.Size([24])
Gradient norm: 1.7259735423139908
tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
torch.Size([24])
Gradient norm: 3.7077303300018785
tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
torch.Size([24])
Gradient norm: 4.593380637474902
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])
torch.Size([24])
Gradient norm: 1.9274313576895652
tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.0051231780591534
tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.987839595695394
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([24])
Gradient norm: 2.885674966793082
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1])
torch.Size([24])
Gradient norm: 3.2200486700635413
tensor([1, 1, 1, 0, 0, 1, 1, 0, 0, 1])
torch.Size([24])
Gradient norm: 1.6240539884150214
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 1.7735231817751125
tensor([1, 1, 

Epoch 3/600 - Training:   0%|          | 0/29 [00:00<?, ?it/s]

...

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 3.267002077165086
tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1])
torch.Size([24])
Gradient norm: 1.9639504503005774
tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
torch.Size([24])
Gradient norm: 2.114356984584047
tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
torch.Size([24])
Gradient norm: 2.4012576269109207
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])
torch.Size([24])
Gradient norm: 2.0419959086992114
tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.3995259402596725
tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.852280778282008
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([24])
Gradient norm: 3.3733210686757813
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1])
torch.Size([24])
Gradient norm: 2.578906119780568
tensor([1, 1, 1, 0, 0, 1, 1, 0, 0, 1])
torch.Size([24])
Gradient norm: 1.6890964761154545
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 2.116764354848153
tensor([1, 1, 0

Epoch 599/600 - Training:   0%|          | 0/29 [00:00<?, ?it/s]

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 4.473581651634564
tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1])
torch.Size([24])
Gradient norm: 2.07324907458245
tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
torch.Size([24])
Gradient norm: 2.2443158871625104
tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
torch.Size([24])
Gradient norm: 2.1792479029065146
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])
torch.Size([24])
Gradient norm: 2.349371133746907
tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.491545217211541
tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.8208146612191234
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([24])
Gradient norm: 3.2317232359264816
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1])
torch.Size([24])
Gradient norm: 2.4673836224875125
tensor([1, 1, 1, 0, 0, 1, 1, 0, 0, 1])
torch.Size([24])
Gradient norm: 1.6389847398483228
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 1.6176399833612054
tensor([1, 1, 0

Epoch 600/600 - Training:   0%|          | 0/29 [00:00<?, ?it/s]

tensor([0, 0, 1, 0, 1, 1, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 3.9051805314146426
tensor([1, 0, 0, 0, 0, 0, 1, 1, 1, 1])
torch.Size([24])
Gradient norm: 2.0205520245579325
tensor([1, 0, 1, 0, 0, 0, 1, 1, 0, 0])
torch.Size([24])
Gradient norm: 2.466817182311929
tensor([1, 0, 0, 1, 1, 1, 0, 0, 1, 0])
torch.Size([24])
Gradient norm: 2.744554214209602
tensor([0, 1, 0, 1, 1, 0, 1, 1, 0, 1])
torch.Size([24])
Gradient norm: 1.652677347018679
tensor([0, 0, 1, 0, 1, 0, 1, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.071578419059116
tensor([1, 0, 0, 0, 1, 0, 0, 1, 1, 0])
torch.Size([24])
Gradient norm: 2.752053842581713
tensor([0, 1, 1, 0, 0, 1, 0, 1, 0, 0])
torch.Size([24])
Gradient norm: 2.9774921367129537
tensor([1, 0, 0, 1, 0, 0, 0, 1, 1, 1])
torch.Size([24])
Gradient norm: 3.103189388041927
tensor([1, 1, 1, 0, 0, 1, 1, 0, 0, 1])
torch.Size([24])
Gradient norm: 1.7674544357943576
tensor([1, 0, 0, 0, 0, 0, 1, 0, 1, 1])
torch.Size([24])
Gradient norm: 2.056018222290116
tensor([1, 1, 0, 

In [22]:
torch.save(model.state_dict(), "techinical_indicator_cnn.pth")

In [23]:
torch.manual_seed(42)
model = CNNTA(1, 80, (6, 6), 2, 2 ).to(device)
model.load_state_dict(torch.load("techinical_indicator_cnn.pth"))
model.to(device)
model.eval()

predicted_val = []
true_val = []
loss = 0
total = 0 

with torch.no_grad():
    
    for batch_x, batch_y in tqdm(dataset_loader_test):
    
        predicted = model(batch_x.to(device))
        values, pred = torch.max(predicted, dim=1)
        
        loss = criterion(predicted, batch_y.to(device))
        loss += batch_x.to(device).size(0)*loss
        total += batch_x.to(device).size(0)
        
        predicted_val.append(pred)
        true_val.append(batch_y.to(device))
        
    aver_loss = loss / total
    print(f"Loss: {aver_loss: .3f}")
all_preds = torch.cat(predicted_val).cpu().numpy()
all_true = torch.cat(true_val).cpu().numpy()

    

TypeError: CNNTA.__init__() missing 1 required positional argument: 'dropout'

In [ ]:
precision = precision_score(all_true, all_preds, average='macro')
recall = recall_score(all_true, all_preds, average='macro')
f1 = f1_score(all_true, all_preds, average='macro')
print(f"Precision: {precision} | Recall: {recall}| f1_score: {f1}")